# Compute and compare different radial potentials

In [1]:
using JAC

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-2464692928112334441\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-17558004737754290893\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-2464692928112334441\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")


We shall here describe how different atomic (radial) potentials can be calculated and applied within the JAC toolbox.

Any (radial) potential is handled in JAC by the data struct `Radial.Potential` that (may) contain a name, the radial function $Z(r)$ as well as the radial grid on which the potential is defined. The potential function `Zr` typically contains the effective charge $Z(r)$ as seen by an electron, so that the full radial potential is given by $V = -\:Z(r) / r$. This convention has been frequently applied in atomic physics and has also been adopted in JAC.


In [2]:
? Radial.Potential

`struct  Radial.Potential`  ... defines a struct for a local radial potential.

```
+ name           ::String            ... A name for the potential.
+ Zr             ::Array{Float64,1}  ... radial potential function Z(r) = - r * V(r) as usual in atomic structure theory.
+ grid           ::RadialGrid        ... radial grid on which the potential is defined.
```

---

`Radial.Potential()`  ... constructor to define an 'empty' instance of the radial potential.



Perhaps, the simplest choice of a radial potential is the **nuclear** potential which, of course, depends on the nuclear model. Let us first simply consider a $\;^{12}$C $\;$ carbon atom with a Fermi-distributed nuclear charge and a standard radial grid.


In [3]:
grid     = Radial.Grid("grid: exponential")
nucModel = Nuclear.Model(6., "Fermi")

Define a radial grid of type MeshGL with 400 grid points
 [rnt=2.000e-06, h=5.000e-02, hp=0.000e+00, NoPoints=390, r_max=9.161e+02;
  B-splines with break points at every 7th point, nsL=56, nsS=57, orderL=7, orderS=8, orderGL=7] 


Fermi nuclear model for Z = 6.0 with mass = 12.18, radius R = 2.4934845703596404 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0.


Then, the corresponding nuclear potential is generated by simply calling:


In [4]:
nucPotential = Nuclear.nuclearPotential(nucModel, grid)

nuclear-potential: Fermi-distributed (radial) potential ... defined on 400 grid points ...
Zr:    [0.0, 0.00368392, 0.0187097, 0.0430085, 0.0723845, 0.101758, 0.126052, 0.141072, 0.149981, 0.171293  …  0.323458, 0.344736, 0.357353, 0.387534, 0.436313, 0.495234, 0.554092, 0.60272, 0.632765, 0.650573]  ...  [6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0  …  6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 390, orderL = 7,  orderS = 8,  nsL = 56,  nsS = 57,  mesh = MeshGL, ...  
r:    [0.0, 2.13272e-8, 1.08316e-7]  ...  [842.031, 887.781, 916.071]
wr:   [0.0, 5.42629e-8, 1.17215e-7]  ...  [52.0386, 38.1203, 17.6472]
t:    [0.0, 0.0, 0.0]  ...  [39.5528, 56.1281, 79.6496]


As seen from the output, the effective charge Z(r) vanishes near to the origin but quickly increases to the full charge $Z = 6.\;$ at larger $r$-values, i.e. near to the surface and outside of the nucleus

To calculate **atomic** potentials, we also need to know the electron density around the nucleus. Most often, this density refers to a particular atomic level. Therefore, we first need to compute some of the low-lying levels of, for instance, neutral carbon. We shall here just compute the levels of the $1s^2 2s^2 2p^2$ ground-state configuration. We use the *simplest* approximation for these computation but will explain in some later tutorials how we could control and refine such simple structure calculations, if needed.


In [5]:
basis     = perform("computation: SCF", [Configuration("1s^2 2s^2 2p^2")], nucModel, grid, AsfSettings())
multiplet = perform("computation: CI",  basis, nucModel, grid, AsfSettings());


... in perform['computation: SCF'] ...
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^0 2p_3/2^2 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^1 2p_3/2^1 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^0 
(Re-) Define a new standard subshell list.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 6.0 with mass = 12.18, radius R = 2.4934845703596404 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.80086331e+01    -1.80086350e+01    +1.02976431e-07    
      2      2s_1/2      -4.50269994e+00    -4.50269857e+00    -3.04312440e-07    
      3      3s_1/2      -2.0


From this multiplet, we just take the *lowest* level (the ground level) in order to calculate the density for the (electronic) potential. In JAC, we have different choices for generating such an **electronic** potential that just arise from the electronic density, and as seen from ? compute at the REPL:

   •    ("radial potential: core-Hartree", grid::Radial.Grid, level::Level) 
       ... to compute a (radial) core-Hartree potential for the given level; 
           cf. Basics.computePotentialCoreHartree. A potential::RadialPotential 
           is returned. 

   •    ("radial potential: Kohn-Sham", grid::Radial.Grid, level::Level) 
       ... to compute a (radial) Kohn-Sham potential for the given level; 
           cf. Basics.computePotentialKohnSham. A potential::RadialPotential is 
           returned.

   •    ("radial potential: Dirac-Fock-Slater", grid::Radial.Grid, level::Level) 
       ... to compute a (radial) Dirac-Fock-Slater potential for the given level; this 
           potential is rather simple but includes some undesired self-interaction and 
           exhibits a wrong asymptotic behaviour. Cf. Basics.computePotentialDFS. 
           A potential::RadialPotential is returned.


In [6]:
level  = multiplet.levels[1]
potCH  = compute("radial potential: core-Hartree", grid, level)
potDFS = compute("radial potential: Dirac-Fock-Slater", grid, level)

coreSubshells = Subshell[1s_1/2, 2s_1/2];    subshells = Subshell[1s_1/2, 2s_1/2, 2p_1/2, 2p_3/2]


DFS (radial) potential ... defined on 400 grid points ...
Zr:    [0.0, -2.02384e-7, -1.04084e-6, -2.39061e-6, -4.02065e-6, -5.65232e-6, -7.00317e-6, -7.83873e-6, -8.3343e-6, -9.51992e-6  …  -1.79821e-5, -1.91662e-5, -1.98685e-5, -2.15492e-5, -2.42681e-5, -2.75562e-5, -3.08447e-5, -3.35646e-5, -3.52462e-5, -3.62435e-5]  ...  [-5.99985, -5.99985, -5.99986, -5.99987, -5.9999, -5.99993, -5.99995, -5.99997, -5.99997, -5.99998  …  -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0, -6.0]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 390, orderL = 7,  orderS = 8,  nsL = 56,  nsS = 57,  mesh = MeshGL, ...  
r:    [0.0, 2.13272e-8, 1.08316e-7]  ...  [842.031, 887.781, 916.071]
wr:   [0.0, 5.42629e-8, 1.17215e-7]  ...  [52.0386, 38.1203, 17.6472]
t:    [0.0, 0.0, 0.0]  ...  [39.5528, 56.1281, 79.6496]


All these potentials are represented as *effective charge* $Z(r)$ and can be added to the nuclear potential, for example:

In [7]:
add(nucPotential, potDFS)

nuclear-potential: Fermi-distributed+DFS (radial) potential ... defined on 400 grid points ...
Zr:    [0.0, 0.00368372, 0.0187087, 0.0430061, 0.0723805, 0.101753, 0.126045, 0.141065, 0.149972, 0.171283  …  0.32344, 0.344717, 0.357334, 0.387512, 0.436288, 0.495206, 0.554061, 0.602687, 0.632729, 0.650537]  ...  [0.000154648, 0.000148665, 0.000143929, 0.000130192, 0.000104555, 7.42171e-5, 4.90205e-5, 3.30865e-5, 2.53332e-5, 2.14125e-5  …  -3.55271e-15, -3.55271e-15, -3.55271e-15, 1.77636e-15, 8.88178e-16, 2.66454e-15, -3.55271e-15, -1.77636e-15, 0.0, 0.0]
Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.0,  NoPoints = 390, orderL = 7,  orderS = 8,  nsL = 56,  nsS = 57,  mesh = MeshGL, ...  
r:    [0.0, 2.13272e-8, 1.08316e-7]  ...  [842.031, 887.781, 916.071]
wr:   [0.0, 5.42629e-8, 1.17215e-7]  ...  [52.0386, 38.1203, 17.6472]
t:    [0.0, 0.0, 0.0]  ...  [39.5528, 56.1281, 79.6496]


As seen from this output, the effective charge of the overall atomic potential goes to *zero* as required for a neutral atom. We can display these potentials also graphically by using `JAC.plot()`

# **The following part of this nootebook is still under construction.**